In [15]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [16]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7.csv')
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
0,36,20033652,이레식당,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,156030,6961.0,NaN,1,NaN,126.937339,37.501398,7호선 장승배기,252.070018,NaN
1,69,20480264,피자스쿨,NaN,Q,음식,Q07,패스트푸드,Q07A01,피자전문,...,150070,7377.0,NaN,NaN,NaN,126.896246,37.502517,7호선 대림(구로구청),655.825278,NaN
2,84,10033391,장터국밥,NaN,Q,음식,Q01,한식,Q01A05,해장국/감자탕,...,132731,1318.0,하,1,8,127.049878,37.684341,7호선 도봉산,490.786535,NaN
3,97,20819907,서울과학기술대학교제2학생식당,NaN,Q,음식,Q14,기타음식업,Q14A01,구내식당/자급식음식점,...,139743,1811.0,NaN,1,NaN,127.076771,37.633069,7호선 공릉(서울과학기술대),651.158559,NaN
4,98,20821938,초밥&참치전문점연,NaN,Q,음식,Q03,일식/수산물,Q03A04,참치전문점,...,143915,5009.0,NaN,NaN,NaN,127.073493,37.546633,7호선 어린이대공원(세종대),59.203796,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12182,135732,17208733,벚꽃스시,NaN,Q,음식,Q03,일식/수산물,Q03A03,음식점-초밥전문,...,156032,6964.0,NaN,1,114,126.942456,37.504310,7호선 장승배기,429.306668,NaN
12183,135741,17228573,생고기김치찌개,NaN,Q,음식,Q01,한식,Q01A02,갈비/삼겹살,...,152815,8348.0,NaN,1,NaN,126.854903,37.488104,7호선 광명사거리,588.584222,NaN
12184,135758,17212010,피굽남,사가정점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,131828,2249.0,NaN,1,NaN,127.089293,37.579951,7호선 용마산,471.931015,NaN
12185,135779,17251264,왕십리곱창,NaN,Q,음식,Q01,한식,Q01A03,곱창/양구이전문,...,131851,2005.0,NaN,NaN,NaN,127.076814,37.612174,7호선 먹골,135.612799,NaN


In [17]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,이레식당,한식,한식/백반/한정식,서울특별시 동작구 성대로31길 71,7호선 장승배기,252.070018,NaN,장승배기,장승배기역 이레식당
1,피자스쿨,패스트푸드,피자전문,서울특별시 영등포구 대림로 247,7호선 대림(구로구청),655.825278,NaN,대림,대림역 피자스쿨
2,장터국밥,한식,해장국/감자탕,서울특별시 도봉구 도봉로180나길 41,7호선 도봉산,490.786535,NaN,도봉산,도봉산역 장터국밥
3,서울과학기술대학교제2학생식당,기타음식업,구내식당/자급식음식점,서울특별시 노원구 공릉로 232,7호선 공릉(서울과학기술대),651.158559,NaN,공릉,공릉역 서울과학기술대학교제2학생식당
4,초밥&참치전문점연,일식/수산물,참치전문점,서울특별시 광진구 능동로 179,7호선 어린이대공원(세종대),59.203796,NaN,어린이대공원,어린이대공원역 초밥&참치전문점연
...,...,...,...,...,...,...,...,...,...
12182,벚꽃스시,일식/수산물,음식점-초밥전문,서울특별시 동작구 상도로30길 40,7호선 장승배기,429.306668,NaN,장승배기,장승배기역 벚꽃스시
12183,생고기김치찌개,한식,갈비/삼겹살,서울특별시 구로구 개봉로11길 18-1,7호선 광명사거리,588.584222,NaN,광명사거리,광명사거리역 생고기김치찌개
12184,피굽남,한식,한식/백반/한정식,서울특별시 중랑구 용마산로51길 45,7호선 용마산,471.931015,NaN,용마산,용마산역 피굽남
12185,왕십리곱창,한식,곱창/양구이전문,서울특별시 중랑구 동일로163길 8,7호선 먹골,135.612799,NaN,먹골,먹골역 왕십리곱창


In [18]:
items = data["검색어"]
items

0                 장승배기역 이레식당
1                   대림역 피자스쿨
2                  도봉산역 장터국밥
3        공릉역 서울과학기술대학교제2학생식당
4          어린이대공원역 초밥&참치전문점연
                ...         
12182             장승배기역 벚꽃스시
12183         광명사거리역 생고기김치찌개
12184               용마산역 피굽남
12185              먹골역 왕십리곱창
12186            태릉입구역 기찻길풍경
Name: 검색어, Length: 12187, dtype: object

In [19]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [20]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/12187
식당 존재
https://place.map.kakao.com/16821812
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/12187
식당 존재
https://place.map.kakao.com/16300615
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/12187
식당 존재 x
진행상황 : 4/12187
식당 존재 x
진행상황 : 5/12187
식당 존재
https://place.map.kakao.com/21086077
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/12187
식당 존재 x
진행상황 : 7/12187
식당 존재
https://place.map.kakao.com/15823847
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/12187
식당 존재
https://place.map.kakao.com/16561932
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/12187
식당 존재
https://place.map.kakao.com/255220684
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/12187
식당 존재 x
진행상황 : 11/12187
식당 존재 x
진행상황 : 12/12187
식당 존재
https://place.map.kakao.com/7882457
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/12187
식당 존재
https://place.map.kakao.com/19970025
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/12187
식당 존재
https://place.map.kakao.com/1584984972
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/12187
식당 존재
https://place.map.k

식당 존재
https://place.map.kakao.com/16082604
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 128/12187
식당 존재
https://place.map.kakao.com/20033052
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 129/12187
식당 존재 x
진행상황 : 130/12187
식당 존재
https://place.map.kakao.com/24664150
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 131/12187
식당 존재
https://place.map.kakao.com/16095636
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 132/12187
식당 존재
https://place.map.kakao.com/16502754
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 133/12187
식당 존재
https://place.map.kakao.com/24902377
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 134/12187
식당 존재
https://place.map.kakao.com/19032274
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 135/12187
식당 존재
https://place.map.kakao.com/16064032
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 136/12187
식당 존재 x
진행상황 : 137/12187
식당 존재
https://place.map.kakao.com/26422211
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 138/12187
식당 존재 x
진행상황 : 139/12187
식당 존재
https://place.map.kakao.com/778181828
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 140/12187

식당 존재
https://place.map.kakao.com/1685826821
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 249/12187
식당 존재
https://place.map.kakao.com/18342597
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 250/12187
식당 존재
https://place.map.kakao.com/21412672
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 251/12187
식당 존재 x
진행상황 : 252/12187
식당 존재
https://place.map.kakao.com/8510748
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 253/12187
식당 존재
https://place.map.kakao.com/10273388
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 254/12187
식당 존재 x
진행상황 : 255/12187
식당 존재
https://place.map.kakao.com/1447245000
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 256/12187
식당 존재 x
진행상황 : 257/12187
식당 존재
https://place.map.kakao.com/1513026476
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 258/12187
식당 존재 x
진행상황 : 259/12187
식당 존재
https://place.map.kakao.com/27312772
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 260/12187
식당 존재 x
진행상황 : 261/12187
식당 존재 x
진행상황 : 262/12187
식당 존재 x
진행상황 : 263/12187
식당 존재
https://place.map.kakao.com/1729417527
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 264/12187


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@id='search.keyword.query']"}
  (Session info: chrome=90.0.4430.93)


In [21]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,12p3llk,대림역 피자스쿨,5점,2021.02.26.
1,7ki33p,대림역 피자스쿨,5점,2019.07.27.
2,,대림역 피자스쿨,5점,2011.02.03.
3,19kn92r,중화역 미스터피자,5점,2020.07.04.
4,1950bi0,중화역 미스터피자,5점,2020.06.13.
...,...,...,...,...
1446,None,어린이대공원역 뉴욕떡볶이,None,2018.03.01.
1447,None,어린이대공원역 뉴욕떡볶이,None,2016.12.11.
1448,,어린이대공원역 뉴욕떡볶이,5점,2017.03.24.
1449,,어린이대공원역 뉴욕떡볶이,5점,2016.10.16.


In [22]:
img_elem

,ItemID,img_url
0,대림역 피자스쿨,None
1,대림역 피자스쿨,None
2,대림역 피자스쿨,None
3,중화역 미스터피자,None
4,중화역 미스터피자,None
...,...,...
1446,어린이대공원역 뉴욕떡볶이,[]
1447,어린이대공원역 뉴욕떡볶이,None
1448,어린이대공원역 뉴욕떡볶이,None
1449,어린이대공원역 뉴욕떡볶이,None


In [14]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,이레식당,한식,한식/백반/한정식,서울특별시 동작구 성대로31길 71,7호선 장승배기,252.070018,https://place.map.kakao.com/16821812,장승배기,장승배기역 이레식당
1,피자스쿨,패스트푸드,피자전문,서울특별시 영등포구 대림로 247,7호선 대림(구로구청),655.825278,https://place.map.kakao.com/16300615,대림,대림역 피자스쿨
2,장터국밥,한식,해장국/감자탕,서울특별시 도봉구 도봉로180나길 41,7호선 도봉산,490.786535,None,도봉산,도봉산역 장터국밥
3,서울과학기술대학교제2학생식당,기타음식업,구내식당/자급식음식점,서울특별시 노원구 공릉로 232,7호선 공릉(서울과학기술대),651.158559,None,공릉,공릉역 서울과학기술대학교제2학생식당
4,초밥&참치전문점연,일식/수산물,참치전문점,서울특별시 광진구 능동로 179,7호선 어린이대공원(세종대),59.203796,https://place.map.kakao.com/21086077,어린이대공원,어린이대공원역 초밥&참치전문점연
...,...,...,...,...,...,...,...,...,...
12182,벚꽃스시,일식/수산물,음식점-초밥전문,서울특별시 동작구 상도로30길 40,7호선 장승배기,429.306668,NaN,장승배기,장승배기역 벚꽃스시
12183,생고기김치찌개,한식,갈비/삼겹살,서울특별시 구로구 개봉로11길 18-1,7호선 광명사거리,588.584222,NaN,광명사거리,광명사거리역 생고기김치찌개
12184,피굽남,한식,한식/백반/한정식,서울특별시 중랑구 용마산로51길 45,7호선 용마산,471.931015,NaN,용마산,용마산역 피굽남
12185,왕십리곱창,한식,곱창/양구이전문,서울특별시 중랑구 동일로163길 8,7호선 먹골,135.612799,NaN,먹골,먹골역 왕십리곱창


In [23]:
img_elem

,ItemID,img_url
0,대림역 피자스쿨,None
1,대림역 피자스쿨,None
2,대림역 피자스쿨,None
3,중화역 미스터피자,None
4,중화역 미스터피자,None
...,...,...
1446,어린이대공원역 뉴욕떡볶이,[]
1447,어린이대공원역 뉴욕떡볶이,None
1448,어린이대공원역 뉴욕떡볶이,None
1449,어린이대공원역 뉴욕떡볶이,None


In [24]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,12p3llk,대림역 피자스쿨,5점,2021.02.26.
1,7ki33p,대림역 피자스쿨,5점,2019.07.27.
2,,대림역 피자스쿨,5점,2011.02.03.
3,19kn92r,중화역 미스터피자,5점,2020.07.04.
4,1950bi0,중화역 미스터피자,5점,2020.06.13.
...,...,...,...,...
1446,None,어린이대공원역 뉴욕떡볶이,None,2018.03.01.
1447,None,어린이대공원역 뉴욕떡볶이,None,2016.12.11.
1448,,어린이대공원역 뉴욕떡볶이,5점,2017.03.24.
1449,,어린이대공원역 뉴욕떡볶이,5점,2016.10.16.


In [25]:
review_elem

,ItemID,review
0,대림역 피자스쿨,최고예요
1,대림역 피자스쿨,깔끔하고 맛있고 친절하심
2,대림역 피자스쿨,테이크아웃 피자전문점! 정말 맛있어서 자주 이용하는 곳~ 저렴하고 친절해요 추천합니...
3,중화역 미스터피자,런치피자부페 갔다가 맛있는 피자 풍성하게 먹었어요. 사람도 많이 없고 한산해서 여유...
4,중화역 미스터피자,요즈음 코로나19도 있었고 6월 들어 날씨도 맘에 들지않고 등등 외부환경이 그리 녹...
...,...,...
1446,어린이대공원역 뉴욕떡볶이,근처에서 나름 소문난맛집
1447,어린이대공원역 뉴욕떡볶이,근처에서 제일 맛있다 순대도 맛있음
1448,어린이대공원역 뉴욕떡볶이,아드님인거 같은데 친절하네요 어머니는 뒷방으로 빠지세요
1449,어린이대공원역 뉴욕떡볶이,너무맛있어요


In [26]:
img_elem.head()

,ItemID,img_url
0,대림역 피자스쿨,None
1,대림역 피자스쿨,None
2,대림역 피자스쿨,None
3,중화역 미스터피자,None
4,중화역 미스터피자,None


In [27]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID         ItemID Rating    Timestamp
1434   v5ckg6  어린이대공원역 뉴욕떡볶이     5점  2021.04.11.
1435  1b6ki12  어린이대공원역 뉴욕떡볶이     5점  2021.03.19.
1436   bjpgo8  어린이대공원역 뉴욕떡볶이     3점  2021.01.19.
1437   8upmed  어린이대공원역 뉴욕떡볶이     1점  2020.11.01.
1438   e4mj58  어린이대공원역 뉴욕떡볶이     5점  2020.06.11.
1439   7kgh4f  어린이대공원역 뉴욕떡볶이     1점  2019.10.09.
1440   e5nqp1  어린이대공원역 뉴욕떡볶이     5점  2019.10.05.
1441   ua5ah9  어린이대공원역 뉴욕떡볶이     5점  2019.09.24.
1442   dotks6  어린이대공원역 뉴욕떡볶이     5점  2019.09.17.
1443   g4nrks  어린이대공원역 뉴욕떡볶이     5점  2019.09.01.
1444  109iik1  어린이대공원역 뉴욕떡볶이     5점  2019.06.20.
1445     None  어린이대공원역 뉴욕떡볶이   None  2018.12.14.
1446     None  어린이대공원역 뉴욕떡볶이   None  2018.03.01.
1447     None  어린이대공원역 뉴욕떡볶이   None  2016.12.11.
1448           어린이대공원역 뉴욕떡볶이     5점  2017.03.24.
1449           어린이대공원역 뉴욕떡볶이     5점  2016.10.16.
1450           어린이대공원역 뉴욕떡볶이     5점  2016.06.28.
             ItemID img_url
1434  어린이대공원역 뉴욕떡볶이    None
1435  어린이대공원역 뉴욕떡볶이    None
1436  어린이대공원역 뉴욕떡볶이    None
1437  

In [28]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_rating(263).csv')

In [29]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_User_ID(263).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_img(263).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_review(263).csv')

In [30]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


99

In [31]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line7_(1)263url.csv')